In [0]:
CREATE OR REFRESH STREAMING LIVE TABLE customers_cdc_landing
as 
select * 
from cloud_files('s3://bucket-ba2509/landing/customers', 'csv', 
  map('cloudFiles.inferColumnTypes', 'false',
  'cloudFiles.schemaLocation', 's3://bucket-ba2509/bronze/customers/schema'));

CREATE OR REFRESH LIVE TABLE customers_cdc_bronze
as 
select DISTINCT
cast( CustomerID as int) as CustomerID,
Country,
current_timestamp() as ingestion_time
from LIVE.customers_cdc_landing; 


-- Create the final customers table
CREATE or REFRESH STREAMING table customers_scd1;
APPLY CHANGES INTO customers_scd1
FROM STREAM(
    Live.customers_cdc_bronze
)
KEYS (CustomerID)
SEQUENCE BY ingestion_time
STORED AS SCD TYPE 1;


-- Step 1: Declare the silver table (empty target)
CREATE STREAMING TABLE customers_scd2
COMMENT "Silver customers with full SCD2 history";

-- Step 2: Apply SCD2 changes from bronze
APPLY CHANGES INTO LIVE.customers_scd2
FROM STREAM(LIVE.customers_scd1)
KEYS (CustomerID)
SEQUENCE BY ingestion_time
COLUMNS * EXCEPT (ingestion_time)
STORED AS SCD TYPE 2;